# Model quantization

In [2]:
from preprocessing import *
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import tensorflow_model_optimization as tfmot
quantize_model = tfmot.quantization.keras.quantize_model
from keras import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D, Conv2D
from keras.layers import MaxPooling1D, MaxPooling2D

In [3]:
df_train = pd.read_csv('../training_data/azimuth_train_dataset.csv', index_col=[0])
df_test = pd.read_csv('../training_data/azimuth_test_dataset.csv', index_col=[0])
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoder.fit([[label] for label in df_train['label']])
X_train, y_train, X_test, y_test = create_whole_dataset(df_train, df_test, encoder)

X_train, X_test = np.transpose(np.array([X_train]), axes=[1, 3, 2, 0]), np.transpose(np.array([X_test]), axes=[1, 3, 2, 0])

np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test)

((628560, 13, 15, 1), (210740, 13, 15, 1), (628560, 36), (210740, 36))

In [6]:
# Fit model
epochs, batch_size, verbose = 3, 32, 1

def create_model(X_train, y_train, X_test, y_test):
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

    # Init model
    model = Sequential()

    # Add layers
    model.add(Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features,1)))
    model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    q_aware_model = quantize_model(model)
    q_aware_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return q_aware_model, history

In [7]:
model, history = create_model(X_train, y_train, X_test, y_test)

Epoch 1/3
19643/19643 [==============================] - 173s 9ms/step - loss: 0.1769 - accuracy: 0.9319
Epoch 2/3
19643/19643 [==============================] - 172s 9ms/step - loss: 0.0872 - accuracy: 0.9662
Epoch 3/3
19643/19643 [==============================] - 172s 9ms/step - loss: 0.0703 - accuracy: 0.9733


In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLaye (None, 13, 15, 1)         3         
_________________________________________________________________
quant_conv2d_2 (QuantizeWrap (None, 11, 13, 64)        771       
_________________________________________________________________
quant_conv2d_3 (QuantizeWrap (None, 9, 11, 64)         37059     
_________________________________________________________________
quant_dropout_1 (QuantizeWra (None, 9, 11, 64)         1         
_________________________________________________________________
quant_max_pooling2d_1 (Quant (None, 4, 5, 64)          1         
_________________________________________________________________
quant_flatten_1 (QuantizeWra (None, 1280)              1         
_________________________________________________________________
quant_dense_2 (QuantizeWrapp (None, 100)              

In [9]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the Lite model.
with open('../models/quant_azimuth_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\tkhor\AppData\Local\Temp\tmp20yguavg\assets


INFO:tensorflow:Assets written to: C:\Users\tkhor\AppData\Local\Temp\tmp20yguavg\assets
